In [36]:
import pandas as pd
import numpy as np
import matplotlib
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [37]:
train = pd.read_csv("train_ready_for_model.csv")

In [38]:
train.head()

,Gender,Occupation,City_Category,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase,Age,Product_ID,0,1,2,3,4+
0,1,10,1,0,3,9.772291,11.823740,8370.0,0,684,0,0,1,0,0
1,1,10,1,0,1,6.000000,14.000000,15200.0,0,2406,0,0,1,0,0
2,1,10,1,0,12,9.772291,11.823740,1422.0,0,868,0,0,1,0,0
3,1,10,1,0,12,14.000000,11.823740,1057.0,0,844,0,0,1,0,0
4,0,16,2,0,8,9.772291,12.876114,7969.0,6,2769,0,0,0,0,1


In [39]:

data_column_category = train.select_dtypes(exclude=[np.number]).columns
data_column_category 

Index([], dtype='object')

In [40]:
test = pd.read_csv("test_ready_for_model.csv")

In [41]:
test.head()

,Gender,Occupation,City_Category,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Age,Product_ID,0,1,2,3,4+
0,0,7,3,1,1,11.0,12.819770,4,1216,0,0,1,0,0
1,0,17,2,0,3,5.0,12.790402,2,1063,1,0,0,0,0
2,1,1,3,1,5,14.0,12.763648,3,2799,0,0,0,0,1
3,1,1,3,1,4,9.0,12.763648,3,1379,0,0,0,0,1
4,1,1,2,0,4,5.0,12.000000,2,535,0,1,0,0,0


In [42]:
orig_test = pd.read_csv("test.csv")
orig_test.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3
0,1000004,P00128942,M,46-50,7,B,2,1,1,11.0,NaN
1,1000009,P00113442,M,26-35,17,C,0,0,3,5.0,NaN
2,1000010,P00288442,F,36-45,1,B,4+,1,5,14.0,NaN
3,1000010,P00145342,F,36-45,1,B,4+,1,4,9.0,NaN
4,1000011,P00053842,F,26-35,1,C,1,0,4,5.0,12.0


In [43]:
result = orig_test[["User_ID","Product_ID"]]
result.head()

,User_ID,Product_ID
0,1000004,P00128942
1,1000009,P00113442
2,1000010,P00288442
3,1000010,P00145342
4,1000011,P00053842


In [44]:
result["Purchase"] = train.Purchase.mean()
result.head()

,User_ID,Product_ID,Purchase
0,1000004,P00128942,9263.968713
1,1000009,P00113442,9263.968713
2,1000010,P00288442,9263.968713
3,1000010,P00145342,9263.968713
4,1000011,P00053842,9263.968713


In [45]:
result.to_csv("mean_result.csv",index=False)

In [46]:
#Define target and ID columns:
target = 'Purchase'
IDcol = ['User_ID']
from sklearn import cross_validation, metrics

def modelfit(alg, dtrain, dtest, predictors, target, IDcol, filename):
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain[target])
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])

    #Perform cross-validation:
    cv_score = cross_validation.cross_val_score(alg, dtrain[predictors], dtrain[target], cv=20, scoring='mean_squared_error')
    cv_score = np.sqrt(np.abs(cv_score))
    
    #Print model report:
    print("\nModel Report")
    print("RMSE : %.4g" % np.sqrt(metrics.mean_squared_error(dtrain[target].values, dtrain_predictions)))
    
    #Predict on testing data:
    dtest[target] = alg.predict(dtest[predictors])
    
    #Export submission file:
    IDcol.append(target)
    submission = pd.DataFrame({ x: dtest[x] for x in IDcol})
    submission.to_csv(filename, index=False)

In [47]:
from sklearn.linear_model import LinearRegression
predictors = [x for x in train.columns if x not in [target]+IDcol]
print("predictors : ",predictors)
# print predictors
alg1 = LinearRegression(normalize=True)
modelfit(alg1, train, test, predictors, target, IDcol, 'LinR_res1.csv')
coef1 = pd.Series(alg1.coef_, predictors).sort_values()
coef1.plot(kind='bar', title='Model Coefficients')

predictors :  ['Gender', 'Occupation', 'City_Category', 'Marital_Status', 'Product_Category_1', 'Product_Category_2', 'Product_Category_3', 'Age', 'Product_ID', '0', '1', '2', '3', '4+']

Model Report
RMSE : 4676


KeyError: 'User_ID'

In [ ]:
# Load libraries
import numpy
from numpy import arange
from matplotlib import pyplot
from pandas import read_csv
from pandas import set_option
from pandas.tools.plotting import scatter_matrix

from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score


from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

import time
import datetime

In [ ]:
array = train.values
X = array[:,0:10]
Y = array[:,10]
validation_size = 0.20
seed = 7
X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=validation_size, random_state=seed)

In [ ]:
# Test options and evaluation metric
num_folds = 10
seed = 7
scoring = 'neg_mean_squared_error'

In [ ]:
# KNN Algorithm tuning

#k_values = numpy.array([1,3,5,7,9,11,13,15,17,19,21])
k_values = numpy.array([1,3,5])
param_grid = dict(n_neighbors=k_values)
model = KNeighborsRegressor()
kfold = KFold(n_splits=num_folds, random_state=seed)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold)
grid_result = grid.fit(X_train, Y_train)

In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))